In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "80%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

Ex. IJ = industrial jobs in control total, SF = total square footage for that sector, Average Job space size is IJ/SF, job space threshold needs to no more than = 0.9 * IJ/SF to get to desired 10% vacancy rate in base year – but it could be smaller, maybe 0.65 * IJ/SF is low range (high density) for determining when we need to redistribute some portion of the jobs. But the remaining space not filled by the threshold adjusted DWS jobs would need to be adjusted to perhaps  1.15 * IJ/SF to make remaining job spaces to be filled by location choice consume more space per job (less dense jobs)

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "results.gdb"]
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [3]:
parcels =r"E:\Projects\REMM-Input-Data-Prep-2019\Parcels\2020-WFRC\Outputs\recompile3.gdb\_22_parcels_split_FINAL"
dws_pts = r'E:\Projects\REMM-Input-Data-Prep-2019\Job-Space-Distribution\Inputs\RegionDWS_REMM.gdb\RegionDWS_Employment'

In [ ]:
parcels_df = pd.DataFrame.spatial.from_featureclass(parcels)
parcels.columns

In [ ]:
d_parcels = parcels_df[(parcels_df['COUNTY_ID']== '11') & (parcels_df['BLDG_SQFT'] > 0)].copy()
sl_parcels = parcels_df[(parcels_df['COUNTY_ID']== '35') & (parcels_df['BLDG_SQFT'] > 0)].copy()
w_parcels = parcels_df[(parcels_df['COUNTY_ID']== '57') & (parcels_df['BLDG_SQFT'] > 0)].copy()
parcels_gt = parcels_df[parcels_df['BLDG_SQFT'] > 0].copy()

In [ ]:
d_sqft_max = d_parcels.groupby('TYPE')['BLDG_SQFT'].max()
sl_sqft_max = sl_parcels.groupby('TYPE')['BLDG_SQFT'].max()
w_sqft_max = w_parcels.groupby('TYPE')['BLDG_SQFT'].max()

d_sqft_min = d_parcels.groupby('TYPE')['BLDG_SQFT'].min()
sl_sqft_min = sl_parcels.groupby('TYPE')['BLDG_SQFT'].min()
w_sqft_min = w_parcels.groupby('TYPE')['BLDG_SQFT'].min()

d_sqft_mean = d_parcels.groupby('TYPE')['BLDG_SQFT'].mean()
sl_sqft_mean = sl_parcels.groupby('TYPE')['BLDG_SQFT'].mean()
w_sqft_mean = w_parcels.groupby('TYPE')['BLDG_SQFT'].mean()

d_sqft_sum = d_parcels.groupby('TYPE')['BLDG_SQFT'].sum()
sl_sqft_sum = sl_parcels.groupby('TYPE')['BLDG_SQFT'].sum()
w_sqft_sum = w_parcels.groupby('TYPE')['BLDG_SQFT'].sum()

In [4]:
dws_pts_df = pd.DataFrame.spatial.from_featureclass(dws_pts)
dws_pts_df.columns


Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1',
       'primary_key', 'input_street', 'input_zone', 'x', 'y', 'score',
       'locator', 'matchAddress', 'standardizedAddress', 'addressGrid',
       'message', 'YRQTR', 'Legal_Name', 'DBA', 'PHYSSADD', 'PHYSSADD2',
       'PHYS_CITY', 'PHYS_ST', 'PHYS_ZIP', 'NAICS', 'OWN', 'COUNTY', 'July',
       'LAT', 'LONG', 'MPO_ID', 'REMMSectorID', 'Located', 'Method',
       'PHYSSADD_corrected', 'PHYS_ZIP_corrected', 'Duplicate', 'REMMSector',
       'Emp2019', 'CO_NAME', 'CITY_NAME', 'DISTSUPER', 'DSUP_NAME', 'DISTLRG',
       'DLRG_NAME', 'DISTMED', 'DMED_NAME', 'TAZID_v9', 'TAZID_v832', 'SHAPE'],
      dtype='object')

In [8]:
# split jobs into industrial and non-industrial
ind_pts = dws_pts_df[dws_pts_df['REMMSectorID'].isin([5, 10])].copy()
non_ind_pts = dws_pts_df[~(dws_pts_df['REMMSectorID'].isin([5, 10]))].copy()

ind_pts.spatial.to_featureclass(location=os.path.join(gdb, 'dws_2019_industrial'),sanitize_columns=False)
non_ind_pts.spatial.to_featureclass(location=os.path.join(gdb, 'dws_2019_nonindustrial'),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Job-Space-Distribution\\Outputs\\results.gdb\\dws_2019_nonindustrial'

In [9]:
# get pts that do not overlap with a remm parcel
ind_pts_lyr = arcpy.MakeFeatureLayer_management(os.path.join(gdb, 'dws_2019_industrial'), 'ind_pts')
non_ind_pts_lyr = arcpy.MakeFeatureLayer_management(os.path.join(gdb, 'dws_2019_nonindustrial'), 'non_ind_pts')  


arcpy.SelectLayerByLocation_management(in_layer=ind_pts_lyr, overlap_type="INTERSECT",
                                       select_features=parcels,
                                       selection_type='NEW_SELECTION',
                                       invert_spatial_relationship='INVERT')

ind_pts_no_parcel = arcpy.conversion.FeatureClassToFeatureClass(in_features=ind_pts_lyr, 
                                                               out_path=gdb, 
                                                               out_name="ind_pts_no_parcel")

arcpy.SelectLayerByLocation_management(in_layer=non_ind_pts_lyr, overlap_type="INTERSECT",
                                       select_features=parcels,
                                       selection_type='NEW_SELECTION',
                                       invert_spatial_relationship='INVERT')


non_ind_pts_no_parcel = arcpy.conversion.FeatureClassToFeatureClass(in_features=non_ind_pts_lyr, 
                                                               out_path=gdb, 
                                                               out_name="non_ind_pts_no_parcel")                                                                                     


In [10]:
# use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
target_features = parcels
join_features = os.path.join(gdb, 'dws_2019_industrial')
output_features = os.path.join(gdb, "parcel_dws_ind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('July')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [11]:
# use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
target_features = parcels
join_features = os.path.join(gdb, 'dws_2019_nonindustrial')
output_features = os.path.join(gdb, "parcel_dws_non_ind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('July')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")